In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
from pathlib import Path
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam






In [ ]:
file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom head on top of the base model
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = BatchNormalization()(x)
x = Dense(50, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
historyTM = model.fit(train_datagen,
                    validation_data=val_datagen,
                    steps_per_epoch = len(train_datagen),
                    validation_steps = 1,
                    epochs = 30, 
                    verbose = 1,
                    # callbacks = [model_checkpoint]
                   ) 

In [ ]:
# Plot the learning curves
acc = historyTM.history['accuracy']
val_acc = historyTM.history['val_accuracy']

loss = historyTM.history['loss']
val_loss = historyTM.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

In [ ]:
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='lower right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [16]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalMaxPooling2D, BatchNormalization, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load data using NumPy
file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 5
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Image data augmentation using TensorFlow's ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Load VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a custom head on top of the base model
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = BatchNormalization()(x)
x = Dense(50, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification with sigmoid activation

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_datagen.flow(np.moveaxis(X_train_tensor.numpy(), 1, -1), y_train_tensor.numpy(), batch_size=batch_size),
    validation_data=val_datagen.flow(np.moveaxis(X_val_tensor.numpy(), 1, -1), y_val_tensor.numpy(), batch_size=batch_size),
    steps_per_epoch=len(X_train_tensor) // batch_size,
    validation_steps=len(X_val_tensor) // batch_size,
    epochs=30,
    verbose=1,
    # callbacks=[model_checkpoint]
)


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (80, 636228))